# 🧬 Módulo 4: Modelado de Proteínas y Docking Molecular
## Actividad 4.2: Modelado de Proteínas por Homología

<div align="center">
  
**Universidad de Caldas - Departamento de Química**  
*Introducción a la Química Computacional (173G7G)*  
**Profesor:** José Mauricio Rodas Rodríguez

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maurorodas/Quimica_computacional_173G7G/blob/main/modulo_04_modelado_proteinas_docking/02_modelado_homologia.ipynb)

</div>

---

## 🎯 Objetivos de Aprendizaje

Al finalizar esta actividad, serás capaz de:
- Comprender los principios del modelado por homología
- Realizar búsquedas de templates con BLAST
- Crear alineamientos secuencia-template de calidad
- Utilizar SWISS-MODEL y MODELLER para construir modelos
- Validar modelos usando gráficos de Ramachandran
- Evaluar la calidad estereoquímica de modelos
- Comparar modelos con templates usando RMSD

---

## 1. Instalación de Dependencias

In [ ]:
!pip install biopython requests py3Dmol matplotlib numpy pandas

In [ ]:
from Bio import SeqIO, Align, pairwise2
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import PDB
import requests
import py3Dmol
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

print("Bibliotecas importadas")

## 2. Secuencia Objetivo (Target)

Comenzamos con la secuencia de la proteína que queremos modelar.

In [ ]:
# Ejemplo: Secuencia de una proteína hipotética
# En la práctica, usarías tu secuencia de interés

target_sequence = """
MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEKAVQVKVKALPDAQFEVVHSLAKWKRQTLGQHDFSAGEGLYTHMKALRPDEDRLSPLHSVYVDQWDWERVMGDGERQFSTLKSTVEAIWAGIKATEAAVSEEFGLAPFLPDQIHFVHSQELLSRYPDLDAKGRERAIAKDLGAVFLVGIGGKLSDGHRHDVRAPDYDDWSTPSELGHAGLNGDILVWNPVLEDAFELSSMGIRVDADTLKHQLALTGDEDRLELEWHQALLRGEMPQTIGGGIGQSRLTMLLLQLPHIGQVQAGVWPAAVRESVPSLL
""".replace("\n", "").strip()

print(f"Longitud de la secuencia: {len(target_sequence)} residuos")
print(f"Primeros 50 residuos: {target_sequence[:50]}")

## 3. Búsqueda de Templates con BLAST

Buscaremos proteínas similares con estructura conocida en el PDB.

In [ ]:
# NOTA: BLAST puede tardar varios minutos
# Para fines educativos, usaremos un ejemplo pre-procesado

def buscar_templates_blast(sequence, max_hits=10):
    """
    Realiza una búsqueda BLAST contra PDB
    ADVERTENCIA: Puede tardar varios minutos
    """
    print("Realizando búsqueda BLAST...")
    print("(Esto puede tardar 2-5 minutos)")
    
    result_handle = NCBIWWW.qblast(
        "blastp",
        "pdb",
        sequence,
        hitlist_size=max_hits
    )
    
    blast_records = NCBIXML.parse(result_handle)
    
    hits = []
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                hit_info = {
                    'pdb_id': alignment.hit_id.split('|')[3],
                    'e_value': hsp.expect,
                    'identity': hsp.identities / hsp.align_length * 100,
                    'alignment_length': hsp.align_length,
                    'query_start': hsp.query_start,
                    'query_end': hsp.query_end
                }
                hits.append(hit_info)
    
    return hits

# Descomenta para ejecutar (toma tiempo)
# templates = buscar_templates_blast(target_sequence)

In [ ]:
# Para este tutorial, usaremos templates de ejemplo
templates_ejemplo = [
    {'pdb_id': '1ABC', 'identity': 45.2, 'e_value': 1e-50},
    {'pdb_id': '2XYZ', 'identity': 38.7, 'e_value': 1e-42},
    {'pdb_id': '3DEF', 'identity': 35.1, 'e_value': 1e-38},
]

print("Templates encontrados:")
for i, template in enumerate(templates_ejemplo, 1):
    print(f"{i}. PDB: {template['pdb_id']} | "
          f"Identidad: {template['identity']:.1f}% | "
          f"E-value: {template['e_value']:.2e}")

## 4. Criterios de Selección de Templates

### Reglas generales:
- **Identidad > 30%**: Modelado confiable
- **Identidad 20-30%**: Zona crepuscular (cuidado)
- **Identidad < 20%**: Poco confiable

### Otros factores:
- Resolución cristalográfica (mejor < 2.5 Å)
- Factor R (mejor < 0.25)
- Cobertura del alineamiento

## 5. Alineamiento Secuencia-Template

In [ ]:
# Secuencia del template (ejemplo)
template_sequence = """
MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEKAVQVKVKALPDAQFEVVHSLAKWKRQTLGQHDFSAGEGLYTHMKALRPDEDRLSPLHSVYVDQWDWERVMGDGERQFSTLKSTVEAIWAGIKATEAAVSEEFGLAPFLPDQIHFVHSQELLSRYPDLDAKGRERAIAKDLGAVFLVGIGGKLSDGHRHDVRAPDYDDWSTPSELGHAGLNGDILVWNPVLEDAFELSSMGIRVDADTLKHQLALTGDEDRLELEWHQALLRGEMPQTIGGGIGQSRLTMLLLQLPHIGQVQAGVWPAAVRESVPSLL
""".replace("\n", "").strip()

# Realizar alineamiento
alignments = pairwise2.align.globalxx(target_sequence[:100], template_sequence[:100])

# Mostrar mejor alineamiento
best_alignment = alignments[0]
print("Mejor alineamiento:")
print(pairwise2.format_alignment(*best_alignment))

## 6. Uso de SWISS-MODEL (Interfaz Web)

SWISS-MODEL es un servidor automático para modelado por homología.

**Pasos:**
1. Ir a https://swissmodel.expasy.org/
2. Pegar tu secuencia
3. El servidor busca templates automáticamente
4. Construye y valida el modelo
5. Descargar el modelo generado

In [ ]:
# Función para enviar trabajo a SWISS-MODEL vía API (requiere registro)
def swiss_model_submit(sequence, email):
    """
    Envía un trabajo de modelado a SWISS-MODEL
    Requiere una cuenta y API key
    """
    # Esto es un ejemplo simplificado
    # En la práctica, usa la API oficial de SWISS-MODEL
    print("Para usar SWISS-MODEL:")
    print("1. Visita https://swissmodel.expasy.org/")
    print("2. Pega tu secuencia")
    print("3. Sigue las instrucciones")
    
swiss_model_submit(target_sequence, "tu_email@example.com")

## 7. Introducción a MODELLER

MODELLER es una herramienta potente para modelado por homología.
Requiere licencia académica gratuita.

In [ ]:
# Instalación de MODELLER (requiere licencia)
# !pip install modeller

# Ejemplo básico de uso de MODELLER
modeller_script = """
from modeller import *
from modeller.automodel import *

env = environ()
env.io.atom_files_directory = ['.']

# Crear el modelo
a = automodel(env,
              alnfile='alineamiento.ali',
              knowns='template',
              sequence='target',
              assess_methods=(assess.DOPE, assess.GA341))

a.starting_model = 1
a.ending_model = 5
a.make()
"""

print("Script ejemplo para MODELLER:")
print(modeller_script)

## 8. Validación del Modelo

### Herramientas de validación:
1. **Gráfico de Ramachandran**: Valida ángulos diedros
2. **PROCHECK**: Calidad estereoquímica
3. **VERIFY3D**: Compatibilidad secuencia-estructura
4. **ERRAT**: Estadísticas de interacciones no-covalentes

In [ ]:
def ramachandran_plot(pdb_file):
    """
    Genera un gráfico de Ramachandran básico
    """
    parser = PDB.PDBParser()
    structure = parser.get_structure('protein', pdb_file)
    
    phi_psi = []
    
    for model in structure:
        for chain in model:
            polypeptides = PDB.PPBuilder().build_peptides(chain)
            for poly in polypeptides:
                angles = poly.get_phi_psi_list()
                for phi, psi in angles:
                    if phi and psi:
                        phi_psi.append((np.degrees(phi), np.degrees(psi)))
    
    if phi_psi:
        phi_values = [p[0] for p in phi_psi]
        psi_values = [p[1] for p in phi_psi]
        
        plt.figure(figsize=(8, 8))
        plt.scatter(phi_values, psi_values, alpha=0.5, s=20)
        plt.xlabel('Phi (φ) grados')
        plt.ylabel('Psi (ψ) grados')
        plt.title('Gráfico de Ramachandran')
        plt.xlim(-180, 180)
        plt.ylim(-180, 180)
        plt.axhline(0, color='gray', linewidth=0.5)
        plt.axvline(0, color='gray', linewidth=0.5)
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # Estadísticas
        total = len(phi_psi)
        favorable = sum(1 for p, s in phi_psi 
                       if (-180 < p < -20 and -60 < s < 50) or
                          (-180 < p < -20 and 90 < s < 180))
        
        print(f"Total de residuos: {total}")
        print(f"En regiones favorables: {favorable} ({favorable/total*100:.1f}%)")

# Ejemplo (necesitas un archivo PDB)
# ramachandran_plot("modelo.pdb")

## 9. Comparación Modelo vs Template

In [ ]:
def calcular_rmsd(pdb1, pdb2, chain='A'):
    """
    Calcula RMSD entre dos estructuras
    """
    parser = PDB.PDBParser(QUIET=True)
    
    structure1 = parser.get_structure('s1', pdb1)
    structure2 = parser.get_structure('s2', pdb2)
    
    # Extraer átomos CA de ambas estructuras
    atoms1 = []
    atoms2 = []
    
    for model in structure1:
        for chain_obj in model:
            if chain_obj.id == chain:
                for residue in chain_obj:
                    if 'CA' in residue:
                        atoms1.append(residue['CA'])
    
    for model in structure2:
        for chain_obj in model:
            if chain_obj.id == chain:
                for residue in chain_obj:
                    if 'CA' in residue:
                        atoms2.append(residue['CA'])
    
    # Superponer estructuras
    super_imposer = PDB.Superimposer()
    min_len = min(len(atoms1), len(atoms2))
    super_imposer.set_atoms(atoms1[:min_len], atoms2[:min_len])
    
    rmsd = super_imposer.rms
    
    print(f"RMSD: {rmsd:.3f} Å")
    return rmsd

# Ejemplo
# rmsd = calcular_rmsd("template.pdb", "modelo.pdb")

## 10. Ejercicios Prácticos

### Ejercicio 1
Usa SWISS-MODEL para modelar una proteína de tu elección. Descarga y analiza el modelo.

### Ejercicio 2
Genera un gráfico de Ramachandran para tu modelo y evalúa su calidad.

### Ejercicio 3
Compara tu modelo con la estructura template usando RMSD.

In [ ]:
# Espacio para tus ejercicios

## Referencias

1. Waterhouse et al. (2018). SWISS-MODEL. *Nucleic Acids Research*
2. Webb & Sali (2016). Comparative Protein Structure Modeling Using MODELLER. *Curr Protoc Bioinformatics*
3. Ramachandran et al. (1963). Stereochemistry of polypeptide chain configurations. *J Mol Biol*

---

## 📚 Recursos Adicionales

### Servidores Web
- [SWISS-MODEL](https://swissmodel.expasy.org/)
- [BLAST](https://blast.ncbi.nlm.nih.gov/)
- [PROCHECK](https://www.ebi.ac.uk/thornton-srv/software/PROCHECK/)

### Documentación
- [MODELLER Manual](https://salilab.org/modeller/manual/)
- [BioPython PDB Module](https://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ)

---

## ✅ Verificación de Aprendizaje

Al finalizar esta actividad deberías ser capaz de:

- ✅ Buscar templates apropiados usando BLAST
- ✅ Construir modelos por homología con SWISS-MODEL
- ✅ Generar gráficos de Ramachandran
- ✅ Calcular RMSD entre estructuras
- ✅ Evaluar la calidad de modelos proteicos

---

<div align="center">

## 🎉 ¡Felicitaciones!

Has completado la **Actividad 4.2: Modelado de Proteínas por Homología**

[![Anterior](https://img.shields.io/badge/⬅️_Actividad_4.1-Fundamentos_de_Proteínas-blue.svg)](01_fundamentos_estructura_proteinas.ipynb)
[![Siguiente](https://img.shields.io/badge/Actividad_4.3_➡️-AlphaFold_y_ESMFold-green.svg)](03_alphafold_esmfold.ipynb)

---

📚 **[Volver al Módulo 4](README.md)** | 🏠 **[Inicio del Curso](../README.md)**

---

**Universidad de Caldas - Departamento de Química**  
*Química Computacional 173G7G*

</div>